# Ejemplo de datos con censura

Datos de consumo de gas LP, ¿cuánto Gas LP consumen los hogares en México?, ¿tiene sustitutos?

## 1. Importación de bibliotecas y datos:

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.iolib.summary2 import summary_col
from tobit import *

# Omiting WARNINGS
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read data
GasLP_df = pd.read_stata('Gas_LP.dta')
GasLP_df

* folioviv: Identificador de la vivienda
* foliohog: Identificador del hogar
* ageb: Área geoestadísstica básica
* est_dis: Estrato de diseño muestral
* upm: Unidad primaria de muestreo
* factor: Factor de expansión
* ubica_geo: Ubicación geográfica
* entidad: Entidad
* cve_ent:               
* Entidad: Entidad
* cve_mun     
* nom_mun: nom_mun
* cve_loc: 
* lat_dec_loc: LAT_DEC
* lon_dec_loc: LON_DEC
* altitud: ALTITUD
* tam_loc: Tamaño de localidad
* rural: Rural=1 según folioviv
* tot_integ: Número de integrantes del hogar
* estufa: Tiene estufa en su casa
* calent_sol: Dispone de calentador solar de agua
* tanque_gas: Dispone de tanque de gas estacionario
* tenencia: Tipo de tenencia de la vivienda
* rentada: Vivienda rentada o prestada
* ing_cor: Ingreso corriente en miles
* ing_deciles: # de decil de ingreso
* ling_cor: Log del ingreso corriente mensual
* ling_cor2: Cuadrado del log del ingreso corriente mensual
* gasto_gaslp: Gasto en gas LP
* p_gaslp: Precio por kg del gas LP
* lgasto_gaslp: log de gasto en gas LP
* lp_gaslp: Log de precio por kg de gaslp
* gasto_elect: Gasto en electricdad
* p_elect: Precio en centavos por Kw/hr de la electricidad
* gasto_gasnat: Gasto en gas natural
* p_gasnat: Precio del gas natural en pesos por GJ
* lp_gasnat: Log del precio por GJ de gas natural
* lp_gasnat_kg: Log del precio por kg equivalnte del gas natural
* gasto_carb: Gasto en carbon
* gasto_lena: Gasto en legna
* gasto_perol: Gasto en petróleo en hogar
* gasto_diesel: Gasto en diesel
* gasto_comb: Gasto en combustible para calentar
* gasto_otroscomb: Gasto en otros combustibles
* acc_gasnat: Dummy, =1 si su localidad tiene acceso a gas natural, =0 si no

Fuente: ENIGH-2016 y otras fuentes del Gobierno Federal

## 2. Análisis regresión

¿El gasto en Gas LP es una variabe censurada?

In [ ]:
#
GasLP_df2 = GasLP_df[GasLP_df.gasto_gasnat == 0]
GasLP_df2

In [ ]:
GasLP_df2.lgasto_gaslp

In [ ]:
# Remplazamos NaN con 0
GasLP_df2['lgasto_gaslp'].fillna(0, inplace=True)

In [ ]:
# Mostramos algunos valores
GasLP_df2.lgasto_gaslp

In [ ]:
# Borramos valores NaN de la muestra
GasLP_df3 = GasLP_df2.dropna(how='any')
GasLP_df3

In [ ]:
# plotting histogram in order to see
plt.hist(GasLP_df3.gasto_gaslp, normed = True, histtype='bar')

#plt.legend()
plt.ylabel('Gasto en Gas LP')
plt.title('Histogram')

plt.show()

In [ ]:
GasLP_df3['cens'] = 0
GasLP_df3.loc[(GasLP_df3.lgasto_gaslp == 0), 'cens'] = -1
GasLP_df3

In [ ]:
# Dfinimos variables
y = GasLP_df3.lgasto_gaslp
x = GasLP_df3[['ling_cor', 'ling_cor2', 'tot_integ', 'lp_gaslp', 'acc_gasnat']]

In [ ]:
y

In [ ]:
# Variable de censura
cens = pd.Series(GasLP_df3.cens)
cens.value_counts()

In [ ]:
# Estimación del modelo Tobit

tr = TobitModel()
tr = tr.fit(x, y, cens, verbose = True)

In [ ]:
# Resultados
tr.intercept_, tr.coef_

In [ ]:
tr.sigma_

In [ ]:
# Adecuaciones a X:
x = sm.add_constant(x)

In [ ]:
# OLS
model = sm.OLS(y, x)
results = model.fit()
print(results.summary())